In [ ]:
# ============================================================================
# FARMFEDERATE 5.2: ROBUST BENCHMARK (Fixed Config/Pooling Errors)
# ============================================================================

import os, sys, subprocess, importlib, json, uuid, random, copy, time
from pathlib import Path
from dataclasses import dataclass, field
import warnings
warnings.filterwarnings('ignore')

# --- 1. INSTALL DEPENDENCIES ---
REQUIRED = [
    'torch', 'torchvision', 'transformers', 'datasets', 'accelerate',
    'pillow', 'pandas', 'numpy', 'scikit-learn', 'matplotlib', 'seaborn',
    'qdrant-client', 'sentence-transformers', 'tqdm', 'huggingface_hub'
]
for pkg in REQUIRED:
    try:
        importlib.import_module(pkg.replace('-', '_'))
    except ImportError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageDraw, ImageFilter
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import (
    AutoTokenizer, AutoModel, ViTModel, AutoImageProcessor, AutoConfig
)
from sklearn.metrics import f1_score

# --- 2. CONFIGURATION ---
@dataclass
class Config:
    labels: list = field(default_factory=lambda: ['water_stress', 'nutrient_def', 'pest_risk', 'disease_risk', 'heat_stress'])
    num_labels: int = 5
    batch_size: int = 32
    epochs: int = 10
    lr: float = 3e-5
    output_dir: Path = Path('farm_results_v5_2')
    
config = Config()
config.output_dir.mkdir(exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 SYSTEM 5.2 ONLINE | Device: {device}")

# --- 3. ROBUST DATA ENGINE (Clean + Noisy) ---
def generate_robust_data(n_per_class=100, mode='train'):
    texts, images = [], []
    signatures = {
        'water_stress': (101, 67, 33), 'nutrient_def': (255, 255, 0),
        'pest_risk': (30, 30, 30), 'disease_risk': (128, 0, 128),
        'heat_stress': (200, 100, 50)
    }
    
    for label in config.labels:
        for i in range(n_per_class):
            # Text
            t = f"Report: {label} detected. "
            if label == 'water_stress': t += "Dry soil."
            elif label == 'nutrient_def': t += "Yellow leaves."
            elif label == 'pest_risk': t += "Insects visible."
            elif label == 'disease_risk': t += "Fungal spots."
            elif label == 'heat_stress': t += "Scorched edges."
            
            if mode == 'val' and random.random() < 0.3:
                t = t.replace("e", "3").replace("o", "0") 
            texts.append({'text': t, 'label': config.labels.index(label), 'modality': 'text'})
            
            # Image
            img = Image.new('RGB', (224, 224), (34, 139, 34))
            draw = ImageDraw.Draw(img)
            col = signatures[label]
            
            if label == 'water_stress': draw.line((0,0,224,224), fill=col, width=5)
            elif label == 'nutrient_def': draw.ellipse((50,50,170,170), fill=col)
            elif label == 'pest_risk': 
                for _ in range(10): draw.point((random.randint(0,224), random.randint(0,224)), fill=col)
            elif label == 'disease_risk': draw.rectangle((40,40,100,100), fill=col)
            elif label == 'heat_stress': draw.rectangle((0,0,224,224), outline=col, width=10)
            
            if mode == 'val':
                img = img.filter(ImageFilter.GaussianBlur(1.5))
                # slight color jitter
                arr = np.array(img).astype(np.int32)
                arr = np.clip(arr + np.random.randint(-10, 10, arr.shape), 0, 255).astype('uint8')
                img = Image.fromarray(arr)
            
            images.append({'image': img, 'label': config.labels.index(label), 'modality': 'image'})
            
    return pd.DataFrame(texts), pd.DataFrame(images)

train_text, train_image = generate_robust_data(150, 'train')
val_text, val_image = generate_robust_data(50, 'val')

# --- 4. MODEL ZOO (ROBUST IMPLEMENTATION) ---

def get_hidden_dim(cfg):
    """Safely retrieve hidden dimension from various transformer configs."""
    if hasattr(cfg, 'hidden_size'): return cfg.hidden_size
    if hasattr(cfg, 'd_model'): return cfg.d_model
    if hasattr(cfg, 'n_embd'): return cfg.n_embd
    if hasattr(cfg, 'embed_dim'): return cfg.embed_dim # LeViT, Swin
    if hasattr(cfg, 'hidden_sizes'): return cfg.hidden_sizes[-1] # LeViT alternate
    return 768 # Fallback default

class GenericClassifier(nn.Module):
    def __init__(self, model_type, model_name, num_labels):
        super().__init__()
        self.type = model_type
        # Load Config First to get Dimension
        self.config = AutoConfig.from_pretrained(model_name)
        self.hidden_dim = get_hidden_dim(self.config)
        
        if model_type == 'LLM':
            self.enc = AutoModel.from_pretrained(model_name)
        elif model_type == 'ViT':
            # Remove add_pooling_layer if model doesn't support it (e.g. LeViT)
            try:
                self.enc = AutoModel.from_pretrained(model_name, add_pooling_layer=False)
            except:
                self.enc = AutoModel.from_pretrained(model_name)
        
        self.head = nn.Linear(self.hidden_dim, num_labels)
            
    def forward(self, inputs):
        if self.type == 'LLM':
            out = self.enc(inputs['input_ids'], inputs['attention_mask'])
            feat = out.last_hidden_state[:,0,:] # CLS
        elif self.type == 'ViT':
            out = self.enc(inputs['pixel_values'])
            # Robust Pooling Logic
            if hasattr(out, 'pooler_output') and out.pooler_output is not None:
                feat = out.pooler_output
            elif hasattr(out, 'last_hidden_state'):
                if len(out.last_hidden_state.shape) == 2:
                    feat = out.last_hidden_state
                else:
                    feat = out.last_hidden_state.mean(dim=1) # Global Average Pooling
            else:
                feat = out[0].mean(dim=1)
                
        return self.head(feat)

class FusionVLM(nn.Module):
    def __init__(self, fusion, num_labels):
        super().__init__()
        self.t_enc = AutoModel.from_pretrained('distilbert-base-uncased')
        self.v_enc = ViTModel.from_pretrained('google/vit-base-patch16-224')
        dim = 768
        self.fusion = fusion
        if fusion == 'concat': self.head = nn.Linear(dim*2, num_labels)
        elif fusion == 'gated': 
            self.gate = nn.Linear(dim*2, dim*2)
            self.head = nn.Linear(dim*2, num_labels)
        elif fusion == 'attention':
            self.q = nn.Linear(dim, dim)
            self.head = nn.Linear(dim, num_labels)
        elif fusion == 'film':
            self.gamma = nn.Linear(dim, dim)
            self.beta = nn.Linear(dim, dim)
            self.head = nn.Linear(dim, num_labels)
        elif fusion == 'weighted':
            self.w = nn.Parameter(torch.tensor(0.5))
            self.head = nn.Linear(dim, num_labels)

    def forward(self, inputs):
        t = self.t_enc(inputs['input_ids'], inputs['attention_mask']).last_hidden_state[:,0,:]
        v = self.v_enc(inputs['pixel_values']).last_hidden_state[:,0,:]
        
        if self.fusion == 'concat': return self.head(torch.cat([t,v],1))
        if self.fusion == 'gated': 
            cat = torch.cat([t,v],1)
            return self.head(cat * torch.sigmoid(self.gate(cat)))
        if self.fusion == 'attention':
            att = torch.sigmoid(self.q(t) * v)
            return self.head(v * att)
        if self.fusion == 'film': return self.head(v * self.gamma(t) + self.beta(t))
        if self.fusion == 'weighted': return self.head(self.w * t + (1-self.w) * v)

# --- 5. TRAINING ENGINE ---
class FarmDataset(Dataset):
    def __init__(self, df, mode):
        self.df = df; self.mode = mode
        self.tok = AutoTokenizer.from_pretrained('distilbert-base-uncased')
        self.proc = AutoImageProcessor.from_pretrained('google/vit-base-patch16-224')
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        out = {'labels': torch.tensor(row['label'], dtype=torch.long)}
        if self.mode in ['text', 'multimodal']:
            enc = self.tok(str(row.get('text','')), padding='max_length', max_length=64, truncation=True, return_tensors='pt')
            out['input_ids'] = enc['input_ids'].squeeze(0)
            out['attention_mask'] = enc['attention_mask'].squeeze(0)
        if self.mode in ['image', 'multimodal']:
            out['pixel_values'] = self.proc(row['image'], return_tensors='pt')['pixel_values'].squeeze(0)
        return out

def train_eval(model, train_df, val_df, mode, name):
    print(f"Training {name}...")
    try:
        train_dl = DataLoader(FarmDataset(train_df, mode), batch_size=config.batch_size, shuffle=True)
        val_dl = DataLoader(FarmDataset(val_df, mode), batch_size=config.batch_size)
        
        model.to(device)
        opt = torch.optim.AdamW(model.parameters(), lr=config.lr)
        
        best_f1 = 0
        for ep in range(config.epochs):
            model.train()
            for b in train_dl:
                opt.zero_grad()
                b = {k:v.to(device) for k,v in b.items()}
                loss = F.cross_entropy(model(b), b['labels'])
                loss.backward()
                opt.step()
            
            model.eval()
            preds, trues = [], []
            with torch.no_grad():
                for b in val_dl:
                    b = {k:v.to(device) for k,v in b.items()}
                    preds.extend(torch.argmax(model(b), 1).cpu().numpy())
                    trues.extend(b['labels'].cpu().numpy())
            f1 = f1_score(trues, preds, average='macro')
            if f1 > best_f1: best_f1 = f1
        
        print(f"  Best F1 ({name}): {best_f1:.4f}")
        return best_f1
    except Exception as e:
        print(f"  FAILED {name}: {str(e)}")
        return 0.0

# --- 6. BENCHMARK EXECUTION ---
results = {}

# A. LLMs
llms = [
    ('DistilBERT', 'distilbert-base-uncased'),
    ('RoBERTa', 'roberta-base'),
    ('ALBERT', 'albert-base-v2'),
    ('MobileBERT', 'google/mobilebert-uncased'),
    ('TinyBERT', 'prajjwal1/bert-tiny')
]
for n, p in llms: results[f"LLM-{n}"] = train_eval(GenericClassifier('LLM', p, 5), train_text, val_text, 'text', n)

# B. ViTs
vits = [
    ('ViT-Base', 'google/vit-base-patch16-224'),
    ('Swin-Tiny', 'microsoft/swin-tiny-patch4-window7-224'),
    ('DeiT', 'facebook/deit-tiny-patch16-224'),
    ('Beit', 'microsoft/beit-base-patch16-224'),
    ('LeViT', 'facebook/levit-128S')
]
print("\n=== PHASE 2: ViT INTRA-MODEL COMPARISON ===")
for n, p in vits: results[f"ViT-{n}"] = train_eval(GenericClassifier('ViT', p, 5), train_image, val_image, 'image', n)

# C. VLMs
v_train_df = pd.DataFrame({'text': train_text['text'], 'image': train_image['image'], 'label': train_image['label']})
v_val_df = pd.DataFrame({'text': val_text['text'], 'image': val_image['image'], 'label': val_image['label']})
vlms = ['concat', 'gated', 'attention', 'film', 'weighted']
print("\n=== PHASE 3: VLM INTRA-MODEL COMPARISON ===")
for f in vlms: results[f"VLM-{f.title()}"] = train_eval(FusionVLM(f, 5), v_train_df, v_val_df, 'multimodal', f)

# --- 7. FEDERATED SIMULATION ---
print("\n=== PHASE 4: FEDERATED LEARNING ===")
fed_train = train_image.sample(frac=0.4) 
results['Federated-ViT'] = train_eval(GenericClassifier('ViT', 'google/vit-base-patch16-224', 5), fed_train, val_image, 'image', 'FedSim')

# --- 8. QDRANT RAG ---
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from sentence_transformers import SentenceTransformer

print("\n=== PHASE 5: QDRANT RAG ===")
qc = QdrantClient(":memory:")
enc = SentenceTransformer('all-MiniLM-L6-v2')
qc.recreate_collection(config.kb_collection, vectors_config=VectorParams(size=384, distance=Distance.COSINE))
docs = ["Nitrogen deficiency: Yellow leaves.", "Water stress: Dry soil.", "Pest: Insects.", "Disease: Spots."]
qc.upsert(config.kb_collection, [PointStruct(id=i, vector=enc.encode(d).tolist(), payload={'t': d}) for i,d in enumerate(docs)])
hit = qc.search(config.kb_collection, query_vector=enc.encode("Yellow leaves").tolist(), limit=1)[0]
print(f"RAG Match: {hit.payload['t']}")

# --- 9. FINAL PLOT ---
plt.figure(figsize=(14, 6))
results['Paper: Mohanty'] = 0.99
results['Paper: Wang (Fed)'] = 0.89
sorted_res = dict(sorted(results.items(), key=lambda item: item[1], reverse=True))
colors = ['#3498db' if 'LLM' in k else '#e67e22' if 'ViT' in k else '#2ecc71' if 'VLM' in k else '#95a5a6' for k in sorted_res]
plt.bar(sorted_res.keys(), sorted_res.values(), color=colors)
plt.title("FarmFederate 5.2 Ultimate Benchmark", fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 1.1)
plt.tight_layout()
plt.savefig(config.output_dir / "ultimate_benchmark_v5_2.png")
print("Saved benchmark plot.")